In [1]:
from dotenv import load_dotenv
import os
import requests
import numpy as np
import spotipy
import time
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data4
import pandas as pd
import random
import credentials as cred

load_dotenv()

True

In [4]:
spotify_token  = cred.spotifyToken()
headers = {"Authorization":f"Bearer {spotify_token}"}

In [5]:
url_base = "https://api.spotify.com/v1/playlists/"
playlist_link = 'https://open.spotify.com/playlist/37i9dQZF1DXcBWIGoYBM5M?si=b705194596334305'

In [6]:
test_query = cred.playlist_query ()

In [7]:
response = cred.playlist_response()

In [8]:
track_items_dict = cred.track_items ()

In [9]:
track_items_dict = cred.track_audio_features ()

In [10]:
track_items_dict = cred.get_lyrics ()

Searching for "Die For You - Remix" by The Weeknd...
Done.
Searching for "Flowers" by Miley Cyrus...
Done.
Searching for "Boy's a liar Pt. 2" by PinkPantheress...
Done.
Searching for "Kill Bill" by SZA...
Done.
Searching for "Escapism." by RAYE...
Done.
Searching for "Creepin' (with The Weeknd & 21 Savage)" by Metro Boomin...
Done.
Searching for "Calm Down (with Selena Gomez)" by Rema...
Done.
Searching for "TQG" by KAROL G...
Done.
Searching for "As It Was" by Harry Styles...
Done.
Searching for "Players" by Coi Leray...
Done.
Searching for "CUFF IT" by Beyoncé...
Done.
Searching for "Anti-Hero" by Taylor Swift...
Done.
Searching for "golden hour" by JVKE...
Done.
Searching for "Shakira: Bzrp Music Sessions, Vol. 53" by Bizarrap...
Done.
Searching for "Unholy (feat. Kim Petras)" by Sam Smith...
Done.
Searching for "Just Wanna Rock" by Lil Uzi Vert...
Done.
Searching for "Love Again" by The Kid LAROI...
Done.
Searching for "Heaven" by Niall Horan...
Done.
Searching for "LET GO" by Cent

In [11]:
track_items_dict = cred.track_sentiment ()

In [12]:
df = pd.DataFrame(track_items_dict)

In [13]:
df.head()

,Name,Artist,Popularity,Song ID,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Lyrics,Positive,Neutral,Negative,Compound
0,Die For You - Remix,The Weeknd,78,7oDd86yk8itslrA9HRP2ki,0.531,0.525,1,-6.500,0,0.0671,0.2320,0.000000,0.4410,0.502,66.900,TranslationsEspañolItalianoDeutschDie For You ...,0.125,0.625,0.250,-0.9982
1,Flowers,Miley Cyrus,100,0yLdNVWF3Srea0uzk55zFn,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,TranslationsEspañolPortuguêsKiswahiliDeutschIt...,0.517,0.439,0.044,0.9998
2,Boy's a liar Pt. 2,PinkPantheress,91,6AQbmUe0Qwf5PZnt4HmTXv,0.696,0.809,5,-8.254,1,0.0500,0.2520,0.000128,0.2480,0.857,132.962,TranslationsPortuguêsTürkçeBoy’s a liar Pt. 2 ...,0.195,0.681,0.124,0.9664
3,Kill Bill,SZA,92,1Qrg8KqiBpW07V7PNxwwwL,0.644,0.735,8,-5.747,1,0.0391,0.0521,0.144000,0.1610,0.418,88.980,TranslationsEspañolPortuguêsItalianoTürkçeDeut...,0.203,0.609,0.188,0.6321
4,Escapism.,RAYE,89,5Z2MiIZ5I3jJvvmeWMLbOQ,0.538,0.742,2,-5.355,1,0.1140,0.1380,0.000047,0.0934,0.250,96.107,TranslationsTürkçeEspañolPortuguêsEscapism. Ly...,0.083,0.823,0.095,-0.7889


In [161]:
spotify_genres = pd.read_excel('../data/spotify_genre_final.xlsx')

spotify_genres = spotify_genres[['Genre', 'Title', 'Artist', 'popularity', 'id', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo']]

spotify_genres.rename(columns = {'Title':'Name', 'popularity': 'Popularity', 'id':'Song ID', 'danceability':'Danceability', 'energy': 'Energy', 'key':'Key', 'loudness':'Loudness', 'mode':'Mode', 'speechiness':'Speechiness', 'acousticness':'Acousticness', 'instrumentalness':'Instrumentalness', 'liveness':'Liveness', 'valence':'Valence', 'tempo':'Tempo'}, inplace = True)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import joblib

# from sklearn.model_selection import train_test_split
# data to be used in the model will be 
data = spotify_genres # pd.read_excel('../data/spotify_genre_final.xlsx')[['Genre', 'Title', 'Artist', 'popularity', 'id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

features = ['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo']

train_features = train_data[features]
train_target = train_data['Genre']

test_features = test_data[features]
test_target = test_data['Genre']

# from sklearn.model_selection import GridSearchCV
# Define the parameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None],
    'max_features': ['sqrt', 'log2']
}

# Initialize a random forest classifier
# from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

# Perform a grid search to find the best hyperparameters
grid_search = GridSearchCV(rfc, param_grid, cv=5)
grid_search.fit(train_features, train_target)

# Print the best hyperparameters and the corresponding accuracy
print(f"Best parameters: {grid_search.best_params_}")
print(f"Accuracy: {grid_search.best_score_}")

# import joblib

rfc = RandomForestClassifier(n_estimators = 100, max_depth = 5, max_features = 'log2')
rfc.fit(train_features, train_target)
joblib.dump(rfc, 'new_genre_mode_spotify_project')

rfc = joblib.load('new_genre_mode_spotify_project')

Best parameters: {'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 200}
Accuracy: 0.48812207353580844


In [166]:
# Prepare the new data

df_audio_features = df[features]

# Make predictions on the new data
predictions = rfc.predict(df_audio_features)
print(predictions)

['pop' 'pop' 'latin' 'edm' 'edm' 'pop' 'latin' 'hiphop' 'rock' 'rap' 'rap'
 'pop' 'rock' 'pop' 'pop' 'rock' 'pop' 'latin' 'hiphop' 'rock' 'latin'
 'pop' 'rock' 'latin' 'latin' 'latin' 'pop' 'rock' 'pop' 'edm' 'pop' 'pop'
 'latin' 'latin' 'r&b' 'pop' 'rap' 'rock' 'latin' 'pop' 'latin' 'latin'
 'pop' 'pop' 'pop' 'latin' 'pop' 'rap' 'rap' 'hiphop']


In [173]:
df['Genre'] = predictions
df['Genre'] = df['Genre'].apply(lambda x: x.capitalize())
df

,Name,Artist,Popularity,Song ID,Danceability,Energy,Key,Loudness,Mode,Speechiness,...,Instrumentalness,Liveness,Valence,Tempo,Lyrics,Positive,Neutral,Negative,Compound,Genre
0,Die For You - Remix,The Weeknd,78,7oDd86yk8itslrA9HRP2ki,0.531,0.525,1,-6.500,0,0.0671,...,0.000000,0.4410,0.5020,66.900,TranslationsEspañolItalianoDeutschDie For You ...,0.125,0.625,0.250,-0.9982,Pop
1,Flowers,Miley Cyrus,100,0yLdNVWF3Srea0uzk55zFn,0.707,0.681,0,-4.325,1,0.0668,...,0.000005,0.0322,0.6460,117.999,TranslationsEspañolPortuguêsKiswahiliDeutschIt...,0.517,0.439,0.044,0.9998,Pop
2,Boy's a liar Pt. 2,PinkPantheress,91,6AQbmUe0Qwf5PZnt4HmTXv,0.696,0.809,5,-8.254,1,0.0500,...,0.000128,0.2480,0.8570,132.962,TranslationsPortuguêsTürkçeBoy’s a liar Pt. 2 ...,0.195,0.681,0.124,0.9664,Latin
3,Kill Bill,SZA,92,1Qrg8KqiBpW07V7PNxwwwL,0.644,0.735,8,-5.747,1,0.0391,...,0.144000,0.1610,0.4180,88.980,TranslationsEspañolPortuguêsItalianoTürkçeDeut...,0.203,0.609,0.188,0.6321,Edm
4,Escapism.,RAYE,89,5Z2MiIZ5I3jJvvmeWMLbOQ,0.538,0.742,2,-5.355,1,0.1140,...,0.000047,0.0934,0.2500,96.107,TranslationsTürkçeEspañolPortuguêsEscapism. Ly...,0.083,0.823,0.095,-0.7889,Edm
5,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,94,2dHHgzDwk4BJdRwy9uXhTO,0.715,0.620,1,-6.005,0,0.0484,...,0.000000,0.0822,0.1720,97.950,New Music Friday 12/02/22 LyricsMetro Boomin -...,0.107,0.777,0.116,0.2053,Pop
6,Calm Down (with Selena Gomez),Rema,92,0WtM2NBVQNNJLh6scP13H8,0.801,0.806,11,-5.206,1,0.0381,...,0.000669,0.1140,0.8020,106.999,New Music Friday 08/26/22 LyricsDJ Khaled - BE...,0.158,0.726,0.117,0.9526,Latin
7,TQG,KAROL G,75,0DWdj2oZMBFSzRsi2Cvfzf,0.720,0.630,4,-3.547,0,0.2770,...,0.000000,0.0936,0.6070,179.974,TranslationsDeutschEnglishTQG Lyrics\nLa que t...,0.027,0.897,0.076,-0.9356,Hiphop
8,As It Was,Harry Styles,89,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,0.0557,...,0.001010,0.3110,0.6620,173.930,TranslationsTürkçeEspañolPortuguês日本語ItalianoΕ...,0.065,0.913,0.021,0.8746,Rock
9,Players,Coi Leray,87,6UN73IYd0hZxLi8wFPMQij,0.954,0.516,6,-5.817,1,0.1600,...,0.000008,0.0504,0.6240,105.001,"TranslationsTürkçePlayers Lyrics\nYeah, 'cause...",0.177,0.760,0.063,0.9873,Rap


In [17]:
import nltk
import spacy
import gensim

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Download spaCy resources
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gerardrius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gerardrius/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [19]:
# Load the song lyrics
#with open('lyrics.txt', 'r') as f:
#    lyrics = f.read()
lyrics = track_items_dict['Lyrics'][0]

# Tokenize the lyrics
tokens = nltk.word_tokenize(lyrics.lower())

# Remove stopwords
stop_words = set(nltk.corpus.stopwords.words('english'))
tokens = [token for token in tokens if not token in stop_words]

# Remove punctuation
tokens = [token for token in tokens if token.isalpha()]


In [22]:
# Create a dictionary from the tokenized lyrics
dictionary = gensim.corpora.Dictionary([tokens])

# Convert the tokenized lyrics into a bag-of-words representation
bow_corpus = [dictionary.doc2bow(tokens)]

# Train an LDA model on the bag-of-words representation
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=1,
    random_state=42
)

# Extract the main topic from the LDA model
# Extract the main topic from the LDA model
topic = lda_model.print_topics()[0][1]

# Split the topic into individual words with proportions
topic_words = topic.split('+')

# Extract the word with the highest proportion
representative_word = max(topic_words, key=lambda x: float(x.split('*')[0])) \
                      .split('*')[1].strip('" ')

# Print the representative word
print('The theme of the song is:', representative_word)


The theme of the song is: would


In [28]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [30]:
# Calculate the average vector for all the words in the topic
topic_words = topic.split("+")
topic_vector = np.zeros((nlp.vocab.vectors.shape[1],), dtype="float32")
for word in topic_words:
    word = word.split("*")[1].strip().strip('"')
    word_vector = nlp.vocab[word].vector
    topic_vector += word_vector
topic_vector /= len(topic_words)

# Find the word closest to the topic vector in the Word Embedding space
representative_word = nlp.vocab.vectors.most_similar(
    topic_vector.reshape(1, -1), n=1
)[0][0]

# Print the representative word
print('The theme of the song is:', representative_word)


ValueError: [E010] Word vectors set to length 0. This may be because you don't have a model installed or loaded, or because your model doesn't include word vectors. For more info, see the docs:
https://spacy.io/usage/models

In [35]:
pd.read_fwf('../data/text.txt')

,[,1.,0.,0..1,0..2,0..3,0..4,0.],"During the period of falling in love, each time that we met and especially when we had not met for a long time.",Unnamed: 9,Unnamed: 10,Unnamed: 11
0,[,0.0,1.0,0.0,0.0,0.0,0.0,0.],When I was involved in a traffic accident.,NaN,NaN,NaN
1,[,0.0,0.0,1.0,0.0,0.0,0.0,0.],When I was driving home after several days of...,NaN,NaN,NaN
2,[,0.0,0.0,0.0,1.0,0.0,0.0,0.],When I lost the person who meant the most to me.,NaN,NaN,NaN
3,[,0.0,0.0,0.0,0.0,1.0,0.0,0.],The time I knocked a deer down - the sight of ...,NaN,NaN,NaN
4,[,0.0,0.0,0.0,0.0,0.0,1.0,0.],When I did not speak the truth.,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7474,[,0.0,0.0,1.0,0.0,0.0,0.0,0.],Two years back someone invited me to be the tu...,seco,d,ear
7475,[,0.0,0.0,0.0,1.0,0.0,0.0,0.],I had taken the responsibility to do something...,ent a,d,elt
7476,[,0.0,0.0,0.0,0.0,1.0,0.0,0.],I was at home and I heard a loud sound of spit...,NaN,NaN,NaN
7477,[,0.0,0.0,0.0,0.0,0.0,1.0,0.],I did not do the homework that the teacher had...,NaN,NaN,NaN


In [78]:
spotify_genres = pd.read_excel('../data/spotify_genre_final.xlsx')

In [82]:
# to plot
spotify_genres['Genre'].value_counts()

pop       1161
rock      1136
latin     1036
hiphop     971
edm        956
rap        926
r&b        731
Name: Genre, dtype: int64

In [83]:
spotify_genres.columns

Index(['Genre', 'Title', 'Album_cover_link', 'Artist', 'duration_ms',
       'explicit', 'id', 'popularity', 'release_date',
       'release_date_precision', 'total_tracks', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

In [87]:
spotify_genres = spotify_genres[['Genre', 'Title', 'Artist', 'popularity', 'id', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo']]

In [132]:
spotify_genres.rename(columns = {'Title':'Name', 'popularity': 'Popularity', 'id':'Song ID', 'danceability':'Danceability', 'energy': 'Energy', 'key':'Key', 'loudness':'Loudness', 'mode':'Mode', 'speechiness':'Speechiness', 'acousticness':'Acousticness', 'instrumentalness':'Instrumentalness', 'liveness':'Liveness', 'valence':'Valence', 'tempo':'Tempo'}, inplace = True)

In [133]:
spotify_genres.head(1)

,Genre,Name,Artist,Popularity,Song ID,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,rock,Baba O'Riley,The Who,75,3qiyyUfYe7CRYLucrPmulD,0.489,0.724,5,-8.367,1,0.0352,0.313,0.185,0.287,0.15,117.292


In [144]:
from sklearn.model_selection import train_test_split

In [145]:
# data to be used in the model will be 
data = spotify_genres # pd.read_excel('../data/spotify_genre_final.xlsx')[['Genre', 'Title', 'Artist', 'popularity', 'id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

In [146]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)

In [147]:
features = ['Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo']

In [148]:
train_features = train_data[features]
train_target = train_data['Genre']

In [149]:
test_features = test_data[features]
test_target = test_data['Genre']

In [150]:
rfc.fit(train_features, train_target)

RandomForestClassifier()

In [151]:
predictions = rfc.predict(test_features)

In [152]:
accuracy = (predictions == test_target).sum() / len(predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.4730250481695568


In [153]:
import joblib

# Save the trained model
joblib.dump(rfc, 'genre_mode_spotify_project')

['genre_mode_spotify_project']

In [154]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None],
    'max_features': ['sqrt', 'log2']
}

# Initialize a random forest classifier
rfc = RandomForestClassifier()

# Perform a grid search to find the best hyperparameters
grid_search = GridSearchCV(rfc, param_grid, cv=5)
grid_search.fit(train_features, train_target)

# Print the best hyperparameters and the corresponding accuracy
print(f"Best parameters: {grid_search.best_params_}")
print(f"Accuracy: {grid_search.best_score_}")


Best parameters: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 200}
Accuracy: 0.4891540652798744


In [157]:
rfc = RandomForestClassifier(n_estimators = 100, max_depth = 5, max_features = 'log2')
rfc.fit(train_features, train_target)
joblib.dump(rfc, 'new_genre_mode_spotify_project')

['new_genre_mode_spotify_project']

In [159]:
# Load the saved model
rfc = joblib.load('new_genre_mode_spotify_project')

# Prepare the new data

new_data = pd.DataFrame({
    'Danceability': [0.5],
    'Energy': [0.5],
    'Key': [1],
    'Loudness': [-6.5],
    'Mode': [0],
    'Speechiness': [0.06],
    'Acousticness': [0.23],
    'Instrumentalness': [0.0],
    'Liveness': [0.44],
    'Valence': [0.5],
    'Tempo': [66.9]
})

# Make predictions on the new data
predictions = rfc.predict(new_data)
print(predictions)


['pop']


PDF Panda R., Malheiro R. & Paiva R. P. (2018). “Novel audio features for music emotion recognition”. IEEE Transactions on Affective Computing (IEEE early access). DOI: 10.1109/TAFFC.2018.2820691.

PDFPanda R., Malheiro R., Paiva R. P. (2018). “Musical Texture and Expressivity Features for Music Emotion Recognition”. 19th International Society for Music Information Retrieval Conference – ISMIR 2018, Paris, France.

In [15]:
emotions_list = ['amusement', 'joy', 'eroticism', 'beauty', 'relaxation', 'sadness', 'dreaminess', 'triumph', 'anxiety', 'scariness', 'annoyance', 'defiance', 'feeling pumped up']
emotions_list

['amusement',
 'joy',
 'eroticism',
 'beauty',
 'relaxation',
 'sadness',
 'dreaminess',
 'triumph',
 'anxiety',
 'scariness',
 'annoyance',
 'defiance',
 'feeling pumped up']

In [17]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string

# Load stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load Porter Stemmer
stemmer = PorterStemmer()

# Load song lyrics
lyrics = df.iloc[1]['Lyrics']

# Remove punctuation and convert to lowercase
lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
lyrics = lyrics.lower()

# Tokenize words
words = word_tokenize(lyrics)

# Remove stop words and stem words
words = [stemmer.stem(word) for word in words if word not in stop_words]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gerardrius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
audio_features_request_base = "https://api.spotify.com/v1/audio-features/"
test_request_here = audio_features_request_base + '2HvvnVkD1kcnaYjgKO5E5c'
audio_features_here = requests.get(test_request_here, headers=headers).json() 

In [27]:
def one_song_feature (song_id):
    headers = cred.query_header (spotify_token)
    audio_features_request_base = "https://api.spotify.com/v1/audio-features/"
    song_request = audio_features_request_base + song_id
    features = requests.get(song_request, headers=headers).json()
    return features

In [30]:
test_features = one_song_feature ('0yLdNVWF3Srea0uzk55zFn')

In [49]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Create Bag-of-Words matrix
vectorizer = CountVectorizer()
X_lyrics = vectorizer.fit_transform([lyrics]).toarray()

In [56]:
X_lyrics
counts = np.sum(X_lyrics, axis=0)
top_indices = np.argsort(counts)[::-1][:9]

top_indices
word_counts = dict(zip(vocabulary, counts))


array([ 9, 39, 36,  4, 79,  2, 43, 78, 42])

In [61]:
test_array = np.array([test_features[feature] for feature in ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])
test_array

array([ 7.07000e-01,  6.81000e-01, -4.32500e+00,  6.68000e-02,
        6.32000e-02,  5.15000e-06,  3.22000e-02,  6.46000e-01,
        1.17999e+02])

In [41]:
print(X_lyrics.shape)
print(np.array([[test_features[feature] for feature in ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]]).shape)

(1, 80)
(1, 9)


In [39]:

# Create feature vector
X = np.concatenate((X_lyrics, np.array([[test_features[feature] for feature in ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]])))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 80 and the array at index 1 has size 9

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define emotions list
emotions_list = ['amusement', 'joy', 'eroticism', 'beauty', 'relaxation', 'sadness', 'dreaminess', 'triumph', 'anxiety', 'scariness', 'annoyance', 'defiance', 'feeling pumped up']

# Load pre-trained random forest classifier
clf = joblib.load('random_forest_classifier.joblib')

# Predict emotion
y_pred = clf.predict([X])[0]

# Print predicted emotion
print(emotions_list[y_pred])

